<a href="https://colab.research.google.com/github/nikhilnair31/RM294---Optimatization-1-Group-Project-1/blob/main/RM294_Optimization_1_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Optimisation Problem**

To formulate the marketing budget allocation problem as a linear program (LP), we need to define decision variables, objective function, and constraints. Let's denote the budget allocation for each medium as follows:

***Variables***:

$x_1$: Budget allocated to Print. </br>
$x_2$: Budget allocated to TV. </br>
$x_3$: Budget allocated to SEO. </br>
$x_4$: Budget allocated to AdWords.</br>
$x_5$: Budget allocated to Facebook.</br>
$x_6$: Budget allocated to LinkedIn.</br>
$x_7$: Budget allocated to Instagram.</br>
$x_8$: Budget allocated to Snapchat.</br>
$x_9$: Budget allocated to Twitter.</br>
$x_{10}$: Budget allocated to Email.</br>

***Objective Function***:
The objective is to maximize the overall ROI: </br>
               <h3><center> $ Maximize &nbsp; &nbsp; ROI_i * x_i \quad \text{for all } i = 1, 2, \ldots, 10\$</center></h3>

***Constraints***:
1. **Budget Constraint**:  $\sum_{i=1}^{10} x_i \leq 10\$
    
2. **TV and Print Constraint**: $x_1 + x_2 \leq x_5 + x_{10}$

3. **Social media and SEO/AdWords Constraint**: $\sum_{i=5}^{9} x_i \geq 2*(x_3 + x_4)\$

4. **Individual Budget Constraints**: $x_i \leq  3 \quad \text{for all } i = 1, 2, \ldots, 10\$

4. **Positivity Constraints**: $x_i \geq 0 \quad \text{for all } i = 1, 2, \ldots, 10\$

## Setup

In [1]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 70.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import gurobipy as gp
import pandas as pd

## Data Loading

In [3]:
# Loading ROI data in CSV
roi_csv_filepath = r'/content/drive/MyDrive/Colab Notebooks/2. Fall/O1/Data/ROI_data.csv'
roi_csv_file_df = pd.read_csv(roi_csv_filepath)

## Question 1, 2, 3 - Estimates from 1st Firm

In [9]:
# Create a Gurobi model
ojMod1 = gp.Model("Budget_Allocation")

Restricted license - for non-production use only - expires 2024-10-28


In [10]:
# Defining decision variables
roi_values_1 = roi_csv_file_df.iloc[0, -10:].tolist()
num_vars = len(roi_values_1)
ojMod1X = ojMod1.addMVar(num_vars, name="x")

In [11]:
# Setting the objective function
obj_coeffs1 = roi_values_1
ojMod1.setObjective(gp.quicksum(obj_coeffs1[i] * ojMod1X[i] for i in range(num_vars)), sense=gp.GRB.MAXIMIZE)

In [12]:
ojMod1.addConstr(gp.quicksum(ojMod1X[i] for i in range(num_vars)) <= 10, name="Budget_Constr")
ojMod1.addConstr(ojMod1X[0] + ojMod1X[1] <= ojMod1X[4] + ojMod1X[9], name="Print_TV_Constr")
ojMod1.addConstr(gp.quicksum(ojMod1X[i] for i in range(4, 9)) >= 2*(ojMod1X[2] + ojMod1X[3]), name="Social_Media_Constr")
for i in range(num_vars):
    ojMod1.addConstr(ojMod1X[i] <= 3, name=f"Budget_Constraint_{i}")

In [13]:
ojMod1.Params.OutputFlag = 0
ojMod1.optimize()

In [14]:
ojMod1.objVal

0.45600000000000007

In [15]:
ojMod1X.x

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

In [37]:
budget_alloc_1 = [ojMod1X[i].x for i in range(num_vars)]

df_1st = pd.DataFrame({
    "Channel": ["Print", "TV", "SEO", "AdWords" ,"Facebook", "LinkedIn", "Instagram", "Snapchat", "Twitter", "Email"],
    "Budget Alloc ($M)": budget_alloc_1
})

# Calculate the total budget and total ROI
budget_1 = sum(budget_alloc_1)
roi_1 = sum(obj_coeffs1[i] * budget_alloc_1[i] for i in range(num_vars))

# Print the DataFrame
print(f"1st Firm's Optimal Budget Allocation\n{df_1st}")
print(f"{'Total Budget':25} ${budget_1:.3f}M")
print(f"{'Total ROI':25} ${roi_1:.3f}M")

1st Firm's Optimal Budget Allocation
     Channel Budget Alloc ($M)
0      Print               0.0
1         TV               3.0
2        SEO               0.0
3    AdWords               1.0
4   Facebook               0.0
5   LinkedIn               0.0
6  Instagram               3.0
7   Snapchat               0.0
8    Twitter               0.0
9      Email               3.0
Total Budget              $10.000M
Total ROI                 $0.456M


## Question 4 - Estimates from 2nd Firm

In [17]:
# Create a Gurobi model
ojMod2 = gp.Model("Budget_Allocation")

In [18]:
# Define decision variables
roi_values_2 = roi_csv_file_df.iloc[1, -10:].tolist()
num_vars = len(roi_values_1)
ojMod2X = ojMod2.addMVar(num_vars, name="x")

In [19]:
# Set the objective function
obj_coeffs2 = roi_csv_file_df.iloc[1, -10:].tolist()
ojMod2.setObjective(gp.quicksum(obj_coeffs2[i] * ojMod2X[i] for i in range(num_vars)), sense=gp.GRB.MAXIMIZE)

In [20]:
ojMod2.addConstr(gp.quicksum(ojMod2X[i] for i in range(num_vars)) <= 10, name="Budget_Constr")
ojMod2.addConstr(ojMod2X[0] + ojMod2X[1] <= ojMod2X[4] + ojMod2X[9], name="Print_TV_Constr")
ojMod2.addConstr(gp.quicksum(ojMod2X[i] for i in range(4, 9)) >= 2*(ojMod2X[2] + ojMod2X[3]), name="Social_Media_Constr")
for i in range(num_vars):
    ojMod2.addConstr(ojMod2X[i] <= 3, name=f"Budget_Constraint_{i}")

In [21]:
ojMod2.Params.OutputFlag = 0
ojMod2.optimize()

In [22]:
ojMod2.objVal

0.45600000000000007

In [23]:
ojMod2X.x

array([3., 0., 0., 1., 3., 3., 0., 0., 0., 0.])

In [38]:
budget_alloc_2 = [ojMod2X[i].x for i in range(num_vars)]

df_2nd = pd.DataFrame({
    "Channel": ["Print", "TV", "SEO", "AdWords" ,"Facebook", "LinkedIn", "Instagram", "Snapchat", "Twitter", "Email"],
    "Budget Alloc ($M)": budget_alloc_2
})

# Calculate the total budget and total ROI
budget_2 = sum(budget_alloc_2)
roi_2 = sum(obj_coeffs2[i] * budget_alloc_2[i] for i in range(num_vars))

# Print the DataFrame
print(f"2nd Firm's Optimal Budget Allocation\n{df_2nd}")
print(f"{'Total Budget':25} ${budget_2:.3f}M")
print(f"{'Total ROI':25} ${roi_2:.3f}M")

2nd Firm's Optimal Budget Allocation
     Channel Budget Alloc ($M)
0      Print               3.0
1         TV               0.0
2        SEO               0.0
3    AdWords               1.0
4   Facebook               3.0
5   LinkedIn               3.0
6  Instagram               0.0
7   Snapchat               0.0
8    Twitter               0.0
9      Email               0.0
Total Budget              $10.000M
Total ROI                 $0.456M


## Question 5 - Allocation Comparison, Combining ROIs and Allocations, Checking for 3rd Constraints Significance

### Comparison

In [25]:
df_1st_and_2nd = pd.merge(df_1st, df_2nd, on='Channel', suffixes=('_df1', '_df2'))
df_1st_and_2nd.rename(
    columns={
      'Budget Alloc ($M)_df1': 'Budget Alloc ($M) 1st Firm',
      'Budget Alloc ($M)_df2': 'Budget Alloc ($M) 2nd Firm'
    },
    inplace=True
)

df_1st_and_2nd['ROI 1st Firm'] = obj_coeffs1
df_1st_and_2nd['ROI 2nd Firm'] = obj_coeffs2

df_1st_and_2nd = df_1st_and_2nd[['Channel', 'ROI 1st Firm', 'Budget Alloc ($M) 1st Firm', 'ROI 2nd Firm', 'Budget Alloc ($M) 2nd Firm']]
df_1st_and_2nd

,Channel,ROI 1st Firm,Budget Alloc ($M) 1st Firm,ROI 2nd Firm,Budget Alloc ($M) 2nd Firm
0,Print,0.031,0.0,0.049,3.0
1,TV,0.049,3.0,0.023,0.0
2,SEO,0.024,0.0,0.024,0.0
3,AdWords,0.039,1.0,0.039,1.0
4,Facebook,0.016,0.0,0.044,3.0
5,LinkedIn,0.024,0.0,0.046,3.0
6,Instagram,0.046,3.0,0.026,0.0
7,Snapchat,0.026,0.0,0.019,0.0
8,Twitter,0.033,0.0,0.037,0.0
9,Email,0.044,3.0,0.026,0.0


> The two budget allocations are different but yield the same optimal value



### Combinations

In [39]:
# Multiply 'ROI 1st Firm' by 'Budget Alloc ($M) 2nd Firm' and sum the results
df_1st_and_2nd['1st_roi_2nd_alloc'] = df_1st_and_2nd['ROI 1st Firm'] * df_1st_and_2nd['Budget Alloc ($M) 2nd Firm']
roi_1st_alloc_2nd = df_1st_and_2nd['1st_roi_2nd_alloc'].sum()
print(f"1st Firm's ROI with 2nd Firm's Allocation: ${roi_1st_alloc_2nd:.3f}M")
print(f"1st Firm's ROI with 1st Firm's Allocation: ${roi_1:.3f}M")
print(f"Difference in ROI: ${(roi_1-roi_1st_alloc_2nd):.3f}M")

1st Firm's ROI with 2nd Firm's Allocation: $0.252M
1st Firm's ROI with 1st Firm's Allocation: $0.456M
Difference in ROI: $0.204M


In [40]:
# Multiply 'ROI 1st Firm' by 'Budget Alloc ($M) 2nd Firm' and sum the results
df_1st_and_2nd['2nd_roi_1st_alloc'] = df_1st_and_2nd['ROI 2nd Firm'] * df_1st_and_2nd['Budget Alloc ($M) 1st Firm']
roi_2nd_alloc_1st = df_1st_and_2nd['2nd_roi_1st_alloc'].sum()
print(f"2nd Firm's ROI with 1st Firm's Allocation: ${roi_2nd_alloc_1st:.3f}M")
print(f"2nd Firm's ROI with 2nd Firm's Allocation: ${roi_2:.3f}M")
print(f"Difference in ROI: ${(roi_2-roi_2nd_alloc_1st):.3f}M")

2nd Firm's ROI with 1st Firm's Allocation: $0.264M
2nd Firm's ROI with 2nd Firm's Allocation: $0.456M
Difference in ROI: $0.192M


### 3rd Constraint Significance

In [28]:
# Create a Gurobi model
ojMod3 = gp.Model("Budget_Allocation")

In [29]:
# Define decision variables
roi_values_1 = roi_csv_file_df.iloc[0, -10:].tolist()
num_vars = len(roi_values_1)

ojMod3X = ojMod3.addMVar(num_vars, name="x")

In [30]:
# Set the objective function
obj_coeffs3 = roi_values_1
ojMod3.setObjective(gp.quicksum(obj_coeffs3[i] * ojMod3X[i] for i in range(num_vars)), sense=gp.GRB.MAXIMIZE)

In [31]:
ojMod3.addConstr(gp.quicksum(ojMod3X[i] for i in range(num_vars)) <= 10, name="Budget_Constr")
ojMod3.addConstr(ojMod3X[0] + ojMod3X[1] <= ojMod3X[4] + ojMod3X[9], name="Print_TV_Constr")
ojMod3.addConstr(gp.quicksum(ojMod3X[i] for i in range(4, 9)) >= 2*(ojMod3X[2] + ojMod3X[3]), name="Social_Media_Constr")

<MConstr () *awaiting model update*>

In [32]:
ojMod3.Params.OutputFlag = 0
ojMod3.optimize()

In [33]:
print(f'{ojMod3.objVal}')

0.46499999999999997


In [34]:
print(f'{ojMod3X.x}')

[0. 5. 0. 0. 0. 0. 0. 0. 0. 5.]


In [41]:
budget_alloc_3 = [ojMod3X[i].x for i in range(num_vars)]

df_3rd = pd.DataFrame({
    "Channel": ["Print", "TV", "SEO", "AdWords" ,"Facebook", "LinkedIn", "Instagram", "Snapchat", "Twitter", "Email"],
    "Budget Alloc ($M)": budget_alloc_3
})

# Calculate the total budget and total ROI
budget_3 = sum(budget_alloc_3)
roi_3 = sum(obj_coeffs3[i] * budget_alloc_3[i] for i in range(num_vars))

# Print the DataFrame
print(f"1st Firm's Optimal Budget Allocation\n{df_3rd}")
print(f"{'Total Budget':25} ${budget_3:.3f}M")
print(f"{'Total ROI':25} ${roi_3:.3f}M")

1st Firm's Optimal Budget Allocation
     Channel Budget Alloc ($M)
0      Print               0.0
1         TV               5.0
2        SEO               0.0
3    AdWords               0.0
4   Facebook               0.0
5   LinkedIn               0.0
6  Instagram               0.0
7   Snapchat               0.0
8    Twitter               0.0
9      Email               5.0
Total Budget              $10.000M
Total ROI                 $0.465M




> Ignoring the third condition, yields different allocations for both 1st and 2nd firm but witht he same the Optimal ROI

